In [74]:
import os
import numpy as np
import pandas as pd
import numpy.random as rnd
import seaborn as sns
from matplotlib import animation
import pymc3 as pm
import arviz as az
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sys import platform
import inspect
import arviz as az

az.style.use('arviz-darkgrid')
az.rcParams['plot.max_subplots'] = 50

In [75]:
# Read the data
#set current working directory to where this file is saved
thisdir = globals()['_dh'][0] + "\\" 
os.chdir(thisdir)

if platform == "darwin": 
    # OS X / Mac
    os.environ["PATH"] += os.pathsep + "/usr/local/bin"
elif platform == "win32":
    # Windows
    pass

#path = "/Users/donaldbrown/Dropbox/department/Classes/DS6014/CourseraBayesianML/Week4MCMC/"
file = "2krolls_plus_data.csv"

data = pd.read_csv(file)
# drop adv due to potential dependence
#data = data.drop("adv", axis=1)
data.head()

,id,roll,adv,basis,type
0,0,11,none,initiative,NaN
1,1,10,none,intimidation,check
2,2,19,disadvantage,strength,save
3,3,8,none,dexterity,check
4,4,14,advantage,intimidation,check


In [76]:
file2 = "TravisRollsDataset.csv"

data2 = pd.read_csv(file2)
data2.head()

,id,season,episode,roll,mod,basis,type,adv,critical
0,2,3,1,13,0,wisdom,save,none,none
1,3,3,1,17,0,investigation,check,none,none
2,4,3,1,4,2,dexterity,save,none,none
3,6,3,2,16,7,melee,attack,none,none
4,7,3,2,16,7,melee,attack,advantage,none


In [77]:
# create a column to delineate between simulated data and Travis' rolls
# 0 is a Travis roll
#deter = [0] * len(data2)
data2["deter"] = 0

# 1 is a generated roll
#deter = [1] * len(data)
data["deter"] = 1

In [78]:
# # make list of column names
# 
# col_names.remove("Unnamed: 0")
# col_names.remove("basis_type")
# col_names

In [79]:
# pick out shared columns
col_names = list(data.columns)
merge1 = data[col_names]
merge2 = data2[col_names]

# merge the datasets
total_data_rodeo = pd.concat([merge1, merge2], axis=0).reset_index()

# shuffle the data 3x in replicable way
rodeo = total_data_rodeo.sample(frac=1, random_state=1234567).reset_index(drop=True)
rodeo = rodeo.sample(frac=1, random_state=57389).reset_index(drop=True)
rodeo = rodeo.sample(frac=1, random_state=98754).reset_index(drop=True)
rodeo = rodeo[col_names]

In [80]:
# put rolls into bins
rodeo["new_rolls"] = pd.cut(rodeo.roll, bins=[0, 12, 20], labels=["0-11", "12-20"])
#rodeo = rodeo.drop("roll", axis=1)

In [81]:
series_vals = [str(x) for x in rodeo['roll'].values.tolist()]
cats = [str(x) for x in list(range(1,21))]

# rodeo['roll'] = pd.Categorical(series_vals, 
#                                    categories= cats,
#                                    ordered=False)
rodeo["basis_type"] = rodeo["basis"].astype(str) + "_" + rodeo["type"].astype(str)

rodeo = rodeo.drop(["basis","type","id"], axis=1)

In [82]:
rodeo_dums = pd.get_dummies(rodeo)
rodeo_dums

,roll,deter,adv_advantage,adv_disadvantage,adv_none,new_rolls_0-11,new_rolls_12-20,basis_type_arcana_check,basis_type_athletics_check,basis_type_charisma_check,...,basis_type_nature_check,basis_type_perception_check,basis_type_persuasion_check,basis_type_ranged_attack,basis_type_sleight of hand_check,basis_type_stealth_check,basis_type_strength_check,basis_type_strength_save,basis_type_thieves’ tools_check,basis_type_wisdom_save
0,9,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,20,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,5,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165,7,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2166,11,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2167,17,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2168,9,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
determination = rodeo["deter"]
rolls = rodeo["roll"]

In [84]:
rodeo_dums.columns

Index(['roll', 'deter', 'adv_advantage', 'adv_disadvantage', 'adv_none',
       'new_rolls_0-11', 'new_rolls_12-20', 'basis_type_arcana_check',
       'basis_type_athletics_check', 'basis_type_charisma_check',
       'basis_type_constitution_check', 'basis_type_constitution_save',
       'basis_type_deception_check', 'basis_type_dexterity_check',
       'basis_type_dexterity_save', 'basis_type_initiative_nan',
       'basis_type_insight_check', 'basis_type_intimidation_check',
       'basis_type_investigation_check', 'basis_type_medicine_check',
       'basis_type_melee_attack', 'basis_type_nature_check',
       'basis_type_perception_check', 'basis_type_persuasion_check',
       'basis_type_ranged_attack', 'basis_type_sleight of hand_check',
       'basis_type_stealth_check', 'basis_type_strength_check',
       'basis_type_strength_save', 'basis_type_thieves’ tools_check',
       'basis_type_wisdom_save'],
      dtype='object')

In [85]:
# # Build the model with priors and run the sampling
# with pm.Model() as chd_model:
    
#     # Intercept term & prior
#     beta0 = pm.Normal("beta0", mu=0, sd=1)
#     # Beta coefficients for predictor variables & priors
#     beta = pm.MvNormal("beta", mu=np.zeros(k), cov=np.eye(k), shape=k)
    
    
#     # Calculate the logit 
#     mu = beta0 + pm.math.dot(X, beta)
#     theta = pm.Deterministic("theta",  pm.invlogit(mu))
#     # Pass the logits to a Bernoulli outcome, with the observed data
#     y_hat = pm.Bernoulli("y_hat", p=theta, observed=y) 
    
#     # Sample
#     trace_main = pm.sample(10000, cores=1, tune=1000, init='adapt_diag', progressbar=True)

In [97]:
## https://docs.pymc.io/notebooks/GLM-linear.html

import pymc3 as pm
from pymc3 import  *

#copy the x and y values into new variables
ycol = rodeo_dums.deter.copy()
#don't include die outcome column
xcols = rodeo_dums.loc[:, "adv_advantage":].copy() 

#collect count values
xcolumns = list(xcols.columns)
coeff_count = len(xcolumns)
die_side_count = rodeo.roll.nunique()

In [ ]:
with pm.Model() as hierarchical_model:

    # Priors for the model parameters
    # Gaussians for the means of the priors of the random intercepts and slopes
    mus = []
    for i in range(coeff_count):
        mus.append(pm.Normal("mu_"+str(i), mu=0., sd=1e5))
    
    # Half-Cauchy for the standard deviations of the priors 
    # of the random intercepts and slopes
    sigmas = []
    for i in range(coeff_count):
        sigmas.append(pm.HalfCauchy("sigma_"+str(i), 1e5))

    # Gaussian priors for random intercepts and slopes
    priors = []
    for i in range(coeff_count):
        priors.append(pm.Normal("prior_"+str(i), mu=mus[i], sd=sigmas[i], shape=die_side_count))
    
    # Linear model, initialize w intercept
    mu = priors[0][rolls - 1] 
    
    for die in range(1,coeff_count):
        mu += priors[die][rolls - 1] * xcols.iloc[:, die]
    
    # Transform outcome to probability
    theta = pm.Deterministic('theta', pm.invlogit(mu))
    y_hat = pm.Bernoulli('y_hat', p=theta, observed=ycol)
    
    # Sample the posterior
    hierarchical_trace = pm.sample(draws=500, step=pm.NUTS(target_accept=0.99), tune=2000)
                                        #1000                                        5000
# with Model() as model:
#     # specify glm and pass in data. The resulting linear model, its likelihood and
#     # and all its parameters are automatically added to our model.
#     glm.linear.LinearComponent(x=xcols, y=ycol, intercept=True)
#     trace = sample(3000, cores=2) # draw 3000 posterior samples using NUTS sampling

Multiprocess sampling (2 chains in 2 jobs)
NUTS: [prior_28, prior_27, prior_26, prior_25, prior_24, prior_23, prior_22, prior_21, prior_20, prior_19, prior_18, prior_17, prior_16, prior_15, prior_14, prior_13, prior_12, prior_11, prior_10, prior_9, prior_8, prior_7, prior_6, prior_5, prior_4, prior_3, prior_2, prior_1, prior_0, sigma_28, sigma_27, sigma_26, sigma_25, sigma_24, sigma_23, sigma_22, sigma_21, sigma_20, sigma_19, sigma_18, sigma_17, sigma_16, sigma_15, sigma_14, sigma_13, sigma_12, sigma_11, sigma_10, sigma_9, sigma_8, sigma_7, sigma_6, sigma_5, sigma_4, sigma_3, sigma_2, sigma_1, sigma_0, mu_28, mu_27, mu_26, mu_25, mu_24, mu_23, mu_22, mu_21, mu_20, mu_19, mu_18, mu_17, mu_16, mu_15, mu_14, mu_13, mu_12, mu_11, mu_10, mu_9, mu_8, mu_7, mu_6, mu_5, mu_4, mu_3, mu_2, mu_1, mu_0]


In [ ]:
# Trace plots
with hierarchical_model:
    az.plot_trace(hierarchical_trace, var_names=xcols.columns)

Below we can compare the probability of 

In [ ]:
# Results in table
with hierarchical_model:
    main_idata = az.from_pymc3(hierarchical_trace)
    
azsum_df = az.summary(main_idata, var_names=xcols.columns, round_to=2)
azsum_df.sort_values(by=["mean"])

In [ ]:
# Forest plots
plt.figure(figsize=(6,14))
with model:
    pm.forestplot(trace, var_names=xcols.columns, combined=True);